If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8002/tree?token=find-site-panos```  

This notebook describes a pipeline to find all Google Street View panoramas with an available depth map within a bounding rectangle.  
The bounding rectangle is specified by providing two [EPSG:4326](https://epsg.io/4326) coordinates that are used as corner points.  
Some code and packages are derived from [Street View DEMs](https://github.com/ccheemeng/streetview-dem).  

For each valid found panorama, its ID (serving as a primary key), latitude, and longitude are written to a ```.csv``` file.  
This ```.csv``` file may be used as an input for [```collect-street-view```](collect-street-view.ipynb).  

If necessary, the output ```.csv``` file may be preprocessed with a GIS software like [QGIS](https://www.qgis.org) to further refine the found panoramas beyond being defined by a rectangle.

In [2]:
target_dir = "data"
lat1 = 1.3574933259463122
lon1 = 103.83393051160616
lat2 = 1.3465604545444212
lon2 = 103.84198706157427
output_filename = "shunfu-coordinates.csv"

In [1]:
import aiocsv
import aiofiles
from aiohttp import ClientSession, ClientTimeout

import geo
import streetview

import asyncio
import csv
from pathlib import Path
import os

In [5]:
async def main(p1, p2, resolution, target_crs, filepath):
    Path(filepath).parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["id", "lat", "lon"])
    sample = geo.Sample(p1, p2, resolution)
    latlon = list(sample.generate_latlon_samples())
    lat = [x[0] for x in latlon]
    lon = [x[1] for x in latlon]
    radius = sample.search_radius()
    await write_panos(lat, lon, radius, target_crs, filepath)

async def write_panos(lat, lon, radius, target_crs, filepath):
    ids = set()
    async with ClientSession(timeout=ClientTimeout(total=None), raise_for_status=True) as session:
        async with aiofiles.open(filepath, 'w+') as afp:
            writer = aiocsv.AsyncWriter(afp)
            await writer.writerow(["id", "lat", "lon"])
            for i in range(len(lat)):
                async for row in get_latlon(session, ids, lat[i], lon[i], radius, target_crs):
                    await writer.writerow(row)

async def get_latlon(session, ids, lat, lon, radius, target_crs):
    pano = await streetview.StreetViewAPI.find_pano_full(session, lat, lon, radius)
    if pano and pano.id not in ids and pano.elevation and pano.depth:
        ids.add(pano.id)
        yield (pano.id, pano.lat, pano.lon)

In [6]:
await main((lat1, lon1), (lat2, lon2), 10, 4326, os.path.join(target_dir, output_filename))

5MyG-bAHFDsnU7OVKLGd2g: index 131320 is out of bounds for axis 0 with size 131320
pbQIEkW4a7rb0bwBtQuYdA: invalid literal for int() with base 2: ''
lApUoRh0xyZBn-1pPR_bKw: invalid literal for int() with base 2: ''
jWbosOkuAMr1Uwe5yYqXRg: index 131544 is out of bounds for axis 0 with size 131544
SvB7ZaCSsFr75E6taW2ZhA: invalid literal for int() with base 2: ''
LEVnfsNVk5hATNEDZglU4Q: index 131304 is out of bounds for axis 0 with size 131304
c3C7FSHQiOWGZ8vbqkY7Rw: invalid literal for int() with base 2: ''
kGcKnYBvzCyQp6x0lXgqXQ: invalid literal for int() with base 2: ''
PP7X-mQ2Yc_h9q0ZE-1-Pg: invalid literal for int() with base 2: ''
PP7X-mQ2Yc_h9q0ZE-1-Pg: invalid literal for int() with base 2: ''
BzW8lY2sCkSS2mK8Z0Xe_g: invalid literal for int() with base 2: ''
gFJyATUaanlEmPNHpCr44Q: invalid literal for int() with base 2: ''
